In [1]:
import socket


HOST = "subeen.com"
PORT = 80

s = socket.socket()
s.connect((HOST, PORT))
command = "GET /files/a.txt HTTP/1.1\nHost: subeen.com\nConnection:close\n\n"
s.send(command.encode())

while True:
    data = s.recv(512)
    if data == b"":
        break
    print(data.decode())

s.close()

HTTP/1.1 200 OK
keep-alive: timeout=5, max=100
content-type: text/plain
last-modified: Tue, 14 Sep 2021 13:18:41 GMT
accept-ranges: bytes
content-length: 84
date: Tue, 18 Apr 2023 19:26:37 GMT
server: LiteSpeed
vary: User-Agent
x-turbo-charged-by: LiteSpeed
connection: close

This is a text file.
Congratulations for making a successful connection!
Great job!!


In [2]:
from socket import *


HOST = "localhost"
PORT = 8080

server_socket = socket(AF_INET, SOCK_STREAM)
server_socket.bind(("localhost", 8080))
server_socket.listen(5)

try:
    server_socket = socket(AF_INET, SOCK_STREAM)
    server_socket.bind((HOST, PORT))
    server_socket.listen(5)

    while True:
        print("Server is ready to receive requests!")
        client_socket, address = server_socket.accept()
        print("Client Address", address)
        request_data = client_socket.recv(1024)
        request_data = request_data.decode()
        print()
        print("Request Data:")
        print(request_data)

        response_data = "HTTP/1.1 200 OK\r\n"
        response_data += "Content-Type: text/html; charset=utf-8\r\n\r\n"
        response_data += "<html><body><h1>Hello World!</h1></body></html>\r\n\r\n"
        response_data = response_data.encode()

        client_socket.sendall(response_data)
        client_socket.shutdown(SHUT_WR)
except Exception as e:
    print(e)

Server is ready to receive requests!
Client Address ('127.0.0.1', 1228)

Request Data:
GET / HTTP/1.1
Host: localhost:8080
Connection: keep-alive
sec-ch-ua: "Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Sec-Fetch-Site: none
Sec-Fetch-Mode: navigate
Sec-Fetch-User: ?1
Sec-Fetch-Dest: document
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US,en;q=0.9,bn-BD;q=0.8,bn;q=0.7
Cookie: __RequestVerificationToken=A0eX8aVs4rlgptQyZOLo71wnL89O7VVLJXva_3_e9qzDylt1ZeuFZbaCQQAX8hLyhxrLB5yFhXDKitnaayhhfervRnjugzIMiuQluS3jrZo1; .AspNetCore.Antiforgery.DjLw96WFxcw=CfDJ8OjpPBrOe2dFgbY8AM9ZhTmCYpoMl1PboyGzvLpfpxM2EFEYaHCnL1KkI8LCDXH50m

KeyboardInterrupt: 

In [2]:
import socket
import threading
import socketserver


class ThreadedTCPRequestHandler(socketserver.BaseRequestHandler):
    def handle(self):
        data = str(self.request.recv(1024), "ascii")
        cur_thread = threading.current_thread()
        response = bytes("{}: {}".format(cur_thread.name, data), "ascii")
        self.request.sendall(response)


class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass


def client(ip, port, message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        sock.sendall(bytes(message, "ascii"))
        response = str(sock.recv(1024), "ascii")
        print("Received: {}".format(response))


if __name__ == "__main__":
    # Port 0 means to select an arbitrary unused port
    HOST, PORT = "localhost", 0

    server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)

    with server:
        ip, port = server.server_address

        # Start a thread with the server
        # that thread will then start one
        # more thread for each request
        server_thread = threading.Thread(target = server.serve_forever)

        # Exit the server thread when the main thread terminates
        server_thread.daemon = True
        server_thread.start()
        print("Server loop running in thread: ", server_thread.name)

        client(ip, port, "Hello World 1")
        client(ip, port, "Hello World 2")
        client(ip, port, "Hello World 3")

        server.shutdown()

Server loop running in thread:  Thread-5 (serve_forever)
Received: Thread-6 (process_request_thread): Hello World 1
Received: Thread-7 (process_request_thread): Hello World 2
Received: Thread-8 (process_request_thread): Hello World 3
